<a href="https://colab.research.google.com/github/nikhilkirtane/LGMVIP-Data-Science-Internship/blob/main/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Advanced Level Task-4

Next Word Prediction using Tensorflow and keras library

Import the python libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Load the text data

In [2]:
with open('1661-0.txt', 'r', encoding='utf-8') as file:
    text = file.read()

preprocessing the text data and convert into lowercase then split into words

In [3]:
cls = text.lower().split() #where cls-convert into lowercase and split

create and perform the tokenization

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cls)
total_words = len(tokenizer.word_index) + 1

create input sequences

In [6]:
input_sequences = []
for i in range(1, len(cls)):
    sequence = cls[i-1:i+1]
    input_sequences.append(sequence)

Convert input sequence into numeric

In [7]:
numeric = tokenizer.texts_to_sequences(input_sequences)

padding sequences into consistent length

In [8]:
msl = max([len(seq) for seq in numeric]) #msl- maximum sequence length
numeric = pad_sequences(numeric, maxlen=msl, padding='pre')

split numeric into input x and label y

In [9]:
X = numeric[:, 0]
y = numeric[:, 1]

Building the RNN model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=msl-1),
    tf.keras.layers.SimpleRNN(150),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Train the model

In [15]:
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
3363/3363 [==============================] - 126s 38ms/step - loss: 4.9034 - accuracy: 0.1640
Epoch 2/10
3363/3363 [==============================] - 126s 37ms/step - loss: 4.7643 - accuracy: 0.1697
Epoch 3/10
3363/3363 [==============================] - 132s 39ms/step - loss: 4.6608 - accuracy: 0.1745
Epoch 4/10
3363/3363 [==============================] - 132s 39ms/step - loss: 4.5810 - accuracy: 0.1781
Epoch 5/10
3363/3363 [==============================] - 132s 39ms/step - loss: 4.5195 - accuracy: 0.1784
Epoch 6/10
3363/3363 [==============================] - 127s 38ms/step - loss: 4.4738 - accuracy: 0.1788
Epoch 7/10
3363/3363 [==============================] - 131s 39ms/step - loss: 4.4419 - accuracy: 0.1789
Epoch 8/10
3363/3363 [==============================] - 129s 38ms/step - loss: 4.4180 - accuracy: 0.1775
Epoch 9/10
3363/3363 [==============================] - 126s 37ms/step - loss: 4.4011 - accuracy: 0.1765
Epoch 10/10
3363/3363 [==============================] 

Generating next word

In [16]:
def generate_next_sentence(seed_text, model, tokenizer, msl, num_words=20):
    generated_text = seed_text
    predictions = []

    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        token_list = pad_sequences([token_list], maxlen=msl-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                predictions.append(output_word)
                break
        generated_text += " " + output_word

    return generated_text, predictions

In [17]:
seed_text = "This eBook is for the use of"
generated_text, predictions = generate_next_sentence(seed_text, model, tokenizer, msl, num_words=5)

print(seed_text)
print("Associated Predictions:", predictions)
print("Generated Text:", generated_text)

This eBook is for the use of
Associated Predictions: ['the', 'same', 'relative', 'position', 'in']
Generated Text: This eBook is for the use of the same relative position in
